# 行列分解による推薦モデルの学習

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Embedding
from tensorflow.keras import Model

/Users/masigara/.local/share/virtualenvs/jupyter-pnVMXwKv/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [10]:
class MatrixFactorization(Model):
    def __init__(self):
        super(MatrixFactorization, self).__init__()
        self.user_factors = Embedding(944, 32)
        self.item_factors = Embedding(1683, 32)
    
    def call(self, user_id, item_id):
        return (self.user_factors(user_id) * self.item_factors(item_id))

# モデルのインスタンスを作成
model = MatrixFactorization()

In [11]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
# train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [13]:
@tf.function
def train_step(user_id, item_id, rating):
    with tf.GradientTape() as tape:
        predictions = model(user_id, item_id)
        loss = loss_object(rating, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
#     train_accuracy(rating, predictions)

In [14]:
@tf.function
def test_step(user_id, item_id, rating):
    predictions = model(user_id, item_id)
    t_loss = loss_object(rating, predictions)

    test_loss(t_loss)
#     test_accuracy(rating, predictions)

In [ ]:
# ds, info = tfds.load("movielens/100k-ratings", with_info=True)
# # info
# ds_batch = ds["train"].batch(1).take(1)

# for i in ds_batch:
#     print(i)

ds = tf.data.experimental.make_csv_dataset("../data/movielens100k.csv", batch_size=8)
EPOCHS = 5

for epoch in range(EPOCHS):
    for batch in ds:
        train_step(batch["user_id"], batch["movie_id"], batch["user_rating"])

    #     for batch in test_ds:
    #         test_step(test_images, test_labels)
    # 
    template = 'Epoch {}, Loss: {}'
    print (template.format(1,train_loss.result()))

    train_loss.reset_states()

In [ ]:
# EPOCHS = 1

# for epoch in range(EPOCHS):
#     train_step(train_ds)

#     for test_images, test_labels in test_ds:
#         test_step(test_images, test_labels)

#     template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
#     print (template.format(epoch+1,
#                          train_loss.result(),
#                          train_accuracy.result()*100,
#                          test_loss.result(),
#                          test_accuracy.result()*100))

#     # 次のエポック用にメトリクスをリセット
#     train_loss.reset_states()
#     train_accuracy.reset_states()
#     test_loss.reset_states()
#     test_accuracy.reset_states()